In [1]:
import numpy as np
from mpg.games import mpg
import networkx as nx

In [138]:
G=mpg.mpg_from_file("data/test01.in")
n=len(G)
A=nx.adjacency_matrix(G,weight=None,nodelist=range(n)).todense()
W=nx.adjacency_matrix(G,nodelist=range(n)).todense()

In [139]:
P=A/A.sum(axis=-1,keepdims=True)

In [140]:
(P*W)@(P*W)

array([[  0. ,   0. ,  12.5,   0. ,  10. ,   0. ,   0. ,   0. ],
       [  0. ,   0. ,   0. , -17.5,   0. ,  -6. ,   0. ,   0. ],
       [  0. ,   0. ,   0. ,   0. ,   0. ,   0. , -17.5,   0. ],
       [  0. ,  -7.5,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ],
       [  0. ,   0. ,   0. ,   0. ,   0. ,   0. ,  -4.5,   0. ],
       [  0. ,  -4.5,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ],
       [  0. ,   0. ,  -7.5,   0. ,  -6. ,   0. ,   0. ,   0. ],
       [  0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ]])

In [141]:
def expected_mean_payoffs(A,W,iters=500):
    E=np.zeros(A.shape[0])
    Z=(A*W).sum(axis=0)
    for k in range(iters):
        E=A.T*E+A.T        
    return E@Z/iters

In [171]:
def expected_mean_payoffs_2(A,W):
    return np.linalg.pinv(np.eye(A.shape[0])-np.linalg.pinv(A))@np.linalg.pinv(A)@(-A*W).sum(axis=0)

In [172]:
from numba import jit

def expected_mean_payoffs_3(A,W,iters=1000):
    n=A.shape[0]
    E=np.zeros(n)
    V=np.arange(n)
    for i in range(iters):
        for k in range(n):
            s=np.random.choice(n,p=A[V[k],:])
            E[k]+=W[V[k],s]
            V[k]=s
    return E/iters

In [173]:
expected_mean_payoffs(P,W)

array([ 0.   ,  4.   ,  0.004,  2.5  ,  0.004,  2.   , -0.02 , -0.02 ])

In [174]:
expected_mean_payoffs_2(P,W)

array([-8.3636447e-16, -2.5000000e+00, -4.5000000e+00, -7.0000000e+00,
       -4.5000000e+00, -6.5000000e+00, -1.7500000e+00, -1.7500000e+00])

In [161]:
expected_mean_payoffs_3(P,W,50)

array([ 0.  , -0.02, -0.1 ,  0.08,  0.06,  0.  , -0.04, -0.02])

In [137]:
P

array([[0. , 0.5, 0. , 0. , 0. , 0.5, 0. , 0. ],
       [0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. ],
       [0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ]])